[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uwsampl/tutorial/blob/master/notebook/02_TVM_Tutorial_Relay.ipynb)

Please run the following block to ensure TVM is setup for *this notebook*, each notebook may have its own runtime.

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    ! gsutil cp "gs://tvm-fcrc-binariesd5fce43e-8373-11e9-bfb6-0242ac1c0002/tvm.tar.gz" /tmp/tvm.tar.gz
    ! mkdir -p /tvm
    ! tar -xf /tmp/tvm.tar.gz --strip-components=4 --directory /tvm
    ! ls -la /tvm
    ! bash /tvm/package.sh
    ! pip install onnx 
    # ! pip install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
    # ! pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl
    # Add TVM to the Python path.
    import sys
    import os
    sys.path.append('/tvm/python')
    sys.path.append('/tvm/topi/python')
    sys.path.append('/tvm/vta/python')
    os.environ['TVM_HOME'] = '/tvm'
else:
    print("Notebook executing locally, skipping Colab setup ...")

In [ ]:
if IN_COLAB:
    ! cd /; git clone https://github.com/uwsampl/relay-aot
    sys.path.append('/relay-aot')

# Relay: an Extensible Deep Learning IR

Last year TVM introduced Relay IR – a second generation high-level IR for deep learning. 

Relay's design comes from a simple insight that the critical difference between regular IRs
and deep learning IRs are the primitive values they manipulate. Relay is designed using well
known insights from the programming languages community coupled with TVM's existing 
infrastructure to provide state of the art performance. 

If you are familiar with ideas from programming languages or existing computation graph
representations, we will connect Relay to your existing knowledge during this tutorial.

We will first cover the design of Relay then elaborate on how one can use it to 
accomplish a wide variety of tasks. This portion of the tutorial is focused  
on Relay, but Relay will be discussed throughout the day day. Relay serves
as the interface layer to TVM.

In [ ]:
import tvm
from tvm import relay
import tvm.relay.testing
from tvm.relay.expr_functor import ExprMutator, ExprVisitor
import torch
import torchvision
import onnx
import numpy
import aot

## Language 

We will first briefly introduce the concepts of Relay below.
You can find a full language specification [here](https://docs.tvm.ai/langref/index.html).

### Variables 

In [ ]:
# A single Relay variable, the string is just a hint
x = relay.var('x')

# A Relay variable with a different dtype, defaults to float32.
x = relay.var('x', dtype='int32')

# A Relay variable with a different shape.
x = relay.var('x', shape=(10, 1))

### Operators

Relay provides high performance operators defined in TVM that implement the primitive operations needed by deep learning applications. Operators can be applied to arguments just like regular Python or C++ functions. Common arithemetic operations are provided both via names and operator overloading.

Variables can be used to construct Relay *expressions* which replace the concept of graphs present in previous frameworks. A Relay expression can be viewed much like a graph with extra functionality as we will see as we go
forward.

In [ ]:
w = relay.op.add(x, x)
print(w)

In [ ]:
z = x + x
print(z)

### Functions

The fundamental packaging of computation in Relay is the function. A function is a combination of a set of inputs
and a Relay expression. Relay functions are no different than ones in programming languages today. They replace named subgraphs.

In [ ]:
f = relay.Function([x], z)
print(f)

### Module

Finally, we can give functions a global name and package many of them together into a module. When we add a function to the module, it will be type checked before hand.

When we print the module, you can see the program annotated with all type information. 

In [ ]:
mod = relay.Module({})
fname = relay.GlobalVar('f')
mod[fname] = f

print(mod)

## Frontends

Relay comes with a variety of frontends and supports most major frameworks, including TensorFlow, PyTorch, MxNet, ONNX, Keras and Caffe2.

Below we provide a couple examples of using these frontends to import models into Relay.

You can find specific tutorials on deploying pretrained models below:  

- [ONNX](https://docs.tvm.ai/tutorials/frontend/from_onnx.html#sphx-glr-tutorials-frontend-from-onnx-py)
- [TensorFlow](https://docs.tvm.ai/tutorials/frontend/from_tensorflow.html#sphx-glr-tutorials-frontend-from-tensorflow-py)
- [Keras](https://docs.tvm.ai/tutorials/frontend/from_keras.html#sphx-glr-tutorials-frontend-from-keras-py)
- [PyTorch](https://tvm.ai/2019/05/30/pytorch-frontend)
- [Caffe2](https://docs.tvm.ai/tutorials/frontend/from_caffe2.html#sphx-glr-tutorials-frontend-from-caffe2-py)

In [ ]:
torch_resnet18 = torchvision.models.resnet18()
dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(torch_resnet18, dummy_input, "resnet.onnx",  export_params=True, verbose=True)

In [ ]:
onnx_resnet18 = onnx.load('resnet.onnx')
func, params = relay.frontend.from_onnx(onnx_resnet18, shape={ '0': (1, 3, 224, 224) })
# print(func)

## Text Format

Relay has a textual representation that can be used to write and print programs. The textual format is still being stablized but can still be of great use today. For example, instead of providing inscrutable graph representations of programs, we can produce human readable output by default.

There are a few different ways to interact with the textual format. The first is to just print out a Relay expression as we have seen above.

In [ ]:
mlp, params = relay.testing.mlp.get_workload(1)
print(mlp)

By default the pretty printer renders the code without metadata. The pretty printer omits metadata because the metadata, which contains information such as constants, is often unreadable. For instance, imagine we perform an optimization, such as inlining the parameters into the program for further optimization. The metadata would include 100s of megabytes of parameters.

In [ ]:
def inline_parameters(expr, params):
    param_map = dict((p.name_hint, p) for p in expr.params)
    params = dict((param_map[k], relay.const(params[k])) for k in params)
    new_body = relay.bind(expr.body, params)
    return relay.Function(relay.ir_pass.free_vars(new_body), new_body)

inline_mlp = inline_parameters(mlp, params)
print(inline_mlp.astext(show_meta_data=True))

Relay's pretty printer also allows users to attach debugging output and metadata to the IR. For example you can see the type information on the example above, but we can also customize the annotation process by passing a callback for annotating nodes. 

In [ ]:
i = 0 
def ann(*args):
    global i
    i += 1
    return f" <expression: {i}>"

print(mlp.astext(show_meta_data=True, annotate=ann))

Finally, an important part of the Relay text format is the ability to load Relay code 
like a normal programming language. We can use the Relay parser to parse code. We actually do this to define the Relay
*prelude* the small standard library of utilities shipped in Relay. 

## Executing Relay

Now that we have looked at how to write and manipulate a Relay program, we will show you how to run one. Relay has multiple execution mechanisms: a custom *debug interpreter* for Relay which can be used for experimentation and debugging, TVM's older graph runtime, the existing execution mechanism, and Relay VM. Relay VM is a newly designed execution mechanism with the goal to smoothly execute all of Relay efficiently. 

We provide a high level interface which imposes some wrapping overhead but enables quick experimentation with each API. 

In [ ]:
mod = relay.Module()
debug_ex = relay.create_executor('debug', mod=mod)
graph_ex = relay.create_executor('graph', mod=mod)
vm_ex = relay.create_executor('vm', mod=mod)

Each executor can be used to evaluate an expression given a Relay module, in this case we use an empty module, and will just evaluate the same expression, a MLP, using each one.

In [ ]:
debug_mlp = debug_ex.evaluate(mlp)
graph_mlp = graph_ex.evaluate(mlp)
vm_mlp = vm_ex.evaluate(mlp)

Each one can be called like a normal Python function with the inputs passed as positional arguments and the parameters as keyword arguments.

In [ ]:
data = numpy.random.rand(1, 1, 28, 28).astype('float32')
print("Debug: ", debug_mlp(data, **params))
print("Graph: ", graph_mlp(data, **params))
print("VM: ", vm_mlp(data, **params))

### Virtual Machine

The Relay virtual machine is worth highlighting, it is a brand new runtime mechanism for Relay which is beginning to stablize. We encourage new users to check it out and provide feedback on its design, perforomance, and more. The VM enables support for non-standard aspects of Relay including control-flow, closures, and data structures.

For more details on the virtual machine see [here](https://github.com/dmlc/tvm/issues/2810).

## Pass Manager

Relay has a flexible and configurable pass manager with an elegant API that be used to easily compose and schedule pass pipelines. We believe an easy-to-configure pipeline is important to enable intelligent exploration between a variety of 


In [ ]:
resnet, params = relay.testing.resnet.get_workload()

The Relay pass infrastructure takes inspiration from both traditional designs from the compiler world as well as APIs from deep learning. In particular our pass manager uses a uniform interface similar to the `Layer` abstraction provided by many deep learning frameworks. Our passes are defined as parametric functions from Module to Module.

For example we define another type of pass, `Sequential` which runs a set of passes serially over a module. Below we construct a compiler pipeline from a set of transformations, we will first type check, perform cosntant folding, eliminate common subexpressions, and the perform layout transformations.

In [ ]:
seq = relay.transform.Sequential([
    relay.transform.InferType(),
    relay.transform.FoldConstant(),
    relay.transform.EliminateCommonSubexpr(),
    relay.transform.AlterOpLayout()
])

Finally we will construct a module using the ResNet as our *main* function. To do so we:

- Use `relay.build_config` to setup the scoped build config, these scoped configs can be used with Python's `with` statement. After the block ends the configuration will be reset to the defaults.
- Use `tvm.target.create` in a scoped fashion as well to set the target to `llvm`.
- We can now optimize the module with this information and get it out.

In [ ]:
mod = relay.Module({"main": resnet})
with relay.build_config(opt_level=3):
    with tvm.target.create("llvm"):
        mod = seq(mod)
        print(mod["main"])
        

## Optimizations

Defining optimizations to transform your program is straight forward and easy to do in Relay.

For example let's define a simple constant evaluator for Relay.

In [ ]:
class IsConstant(ExprVisitor):
    def __init__(self):
        self.is_const = True
        super().__init__()
    
    def visit_const(self, const):
        self.is_const = self.is_const and True
    
    def visit_op(self, op):
        self.is_const = self.is_const and True
        
    def visit_call(self, call):
        for arg in call.args:
            self.visit(arg)
        
        self.visit(call.op)
        
    def visit(self, expr):
        if not isinstance(expr, (relay.Op, relay.Call, relay.Constant)):
            self.is_const = False
        
        return super().visit(expr)
    
def is_const(expr):
    isc = IsConstant()
    isc.visit(expr)
    return isc.is_const

In [ ]:
class ConstantFolder(ExprMutator):
    def visit_call(self, call):
        if is_const(call):
            ex = relay.create_executor('debug')
            cexpr = ex.evaluate(call)
            return relay.const(cexpr.asnumpy())
        else:
            return super().visit_call(call)

def const_fold(expr):
    return ConstantFolder().visit(expr)

In [ ]:
c1 = relay.const(numpy.ones((10, 1), 'float32'))
c2 = relay.const(numpy.ones((1, 1), 'float32'))
e = c1 + c2
print("Input Expr: ", e)
const = const_fold(e)
assert isinstance(const, relay.Constant)
print("Output: ", const.data)

## Quantization

Relay supports an automatic quantization framework which can be used to implement a variety of quantization schemes. You can find more details about it in our recent paper as well as it being used in action here.

Below we will apply the default quantization scheme to ResNet-18.

In [ ]:
resnet, params = relay.testing.resnet.get_workload()

with relay.quantize.qconfig():
    qresnet = relay.quantize.quantize(resnet, params)
    print(qresnet)

## Heterogeneous Execution

Relay supports a high-level interface for scheduling computation across multiple heterogeneous devices. An interesting property of this pass is that it is not special. It is built using Relay's standard machinery for
passes. 

We implement this by using an annotation to mark which computations we would like to schedule on each device, 
and a pass inserts all the appropriate calls to synchronize memory across devices. 

The below pass uses this machinery to schedule all convolutions onto the GPU.

In [ ]:
class ScheduleConv2d(ExprMutator):
    def __init__(self, device):
        self.device = device
        super().__init__()

    def visit_call(self, expr):
        visit = super().visit_call(expr)
        if expr.op == tvm.relay.op.get("nn.conv2d"):
            return relay.annotation.on_device(visit, self.device)
        else:
            return visit

def schedule_conv2d_on_gpu(expr):
    sched = ScheduleConv2d(tvm.gpu(0))
    return sched.visit(expr)

We can grab a model, we provide a few basic models in Relay's testing library. By default when printing a model we will see it rendered in Relay's textual format.

In [ ]:
# We can grab a model, we provide a few basic models in Relay's testing library.
resnet, params = relay.testing.resnet.get_workload()
print(resnet)

We can now run the customized pass we defined above to schedule individual convolutions on the GPU.

In [ ]:
resnet = schedule_conv2d_on_gpu(resnet)
print(resnet)


We can later rewrite away the device annotations to insert the copies.

In [ ]:
resnet = relay.ir_pass.rewrite_annotated_ops(resnet, 0)
print(resnet)

Finally we will look at a couple case studies of what can be built using Relay. We will first look at how Relay is used as a backend in PyTorch integration, then how Relay can be used to compile a model down to traditional hardware, and finally how it can be used to support a custom accelerator, VTA, which we will dicuss in detail today.

## Ahead of time compilation

An example of what can be built using Relay can be found with the [ahead of time compiler](https://github.com/uwsampl/relay-aot). 

We have already installed the compiler above, we can simply import it and use it below.

In [ ]:
mlp, params = relay.testing.mlp.get_workload(1)
cmlp = aot.compile(mlp, relay.Module({}), ctx=tvm.cpu(0), tgt=tvm.target.create('llvm'))
data = numpy.ones((1, 1, 28, 28)).astype('float32')
print(cmlp(data, *params.values()))

## PyTorch Integration

Recently Facebook engineers have begun to integrate TVM into PyTorch. The are using PyTorch's JIT functionality to generate Relay code which is then optimized and deployed. This support is currently being integrated into mainline TVM, and installed from [here](https://github.com/pytorch/tvm). 

Initial results are promising and a recent writeup on the design can be found on our [blog](https://tvm.ai/2019/05/30/pytorch-frontend). 

![results](https://i.imgur.com/KfJ7oas.png)


```python
def add(a, b, c):
    return a + b + c

# via tracing
relay_graph = torch_tvm.to_relay(add, inputs)

@torch.jit.script
def mul(a, b, c):
    return a * b * c

# via script
relay_graph = torch_tvm.to_relay(mul, inputs)
```